In [ ]:
import tensorflow_probability as tfp
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


# Generate synthetic data
np.random.seed(0)
true_mean = 3.0
true_stddev = 1.0
data = np.random.normal(loc=true_mean, scale=true_stddev, size=50).astype(np.float32)

# Define prior
prior = tfp.distributions.Normal(loc=0.0, scale=1.0)

# Define likelihood function
def likelihood_fn(mean):
    likelihood = tfp.distributions.Normal(loc=mean, scale=1.0)
    return tf.reduce_sum(likelihood.log_prob(data))

# Posterior using MCMC
hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=lambda mean: prior.log_prob(mean) + likelihood_fn(mean),
    step_size=0.1,
    num_leapfrog_steps=3,
)

# Initial state
initial_state = tf.constant(0.0)

# Sample posterior
samples, _ = tfp.mcmc.sample_chain(
    num_results=500,
    current_state=initial_state,
    kernel=hmc,
    num_burnin_steps=200,
)

# Generate parameter ranges from MCMC samples
x_posterior = np.linspace(np.min(samples), np.max(samples), 100)

# Plotting prior, likelihood, and posterior
plt.figure(figsize=(12, 6))

# Prior plot
x_prior = np.linspace(-5, 5, 100)
prior_pdf = prior.prob(x_prior)
plt.plot(x_prior, prior_pdf, label="Prior (Initial Belief)")

# Likelihood for actual data
x_likelihood = np.linspace(np.min(data) - 1, np.max(data) + 1, 100)
likelihood_pdf = np.array([tfp.distributions.Normal(loc=mean, scale=1.0).prob(data).numpy().sum() for mean in x_likelihood])
plt.plot(x_likelihood, likelihood_pdf, label="Likelihood (Data Fit)")

# Posterior distribution
posterior_pdf = tfp.distributions.Normal(loc=np.mean(samples), scale=np.std(samples)).prob(x_posterior)
plt.plot(x_posterior, posterior_pdf, label="Posterior (Updated Belief)")

# Show plot with legends
plt.legend()
plt.xlabel("Parameter Value")
plt.ylabel("Probability Density")
plt.title("Prior, Likelihood, and Posterior Distributions")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

# Define prior (Uniform distribution for probability of heads)
prior = tfp.distributions.Uniform(low=0., high=1.)

# Define likelihood function (Binomial: 7 heads out of 10 tosses)
def likelihood_fn(p):
    return tfp.distributions.Binomial(total_count=10, probs=p).prob(7).numpy()

# Define posterior (using Bayes' rule approximation)
p_values = np.linspace(0, 1, 100)
prior_values = prior.prob(p_values).numpy()
likelihood_values = np.array([likelihood_fn(p) for p in p_values])
posterior_values = prior_values * likelihood_values
posterior_values /= posterior_values.sum()  # Normalize

# Plot Prior, Likelihood, and Posterior distributions
plt.figure(figsize=(12, 6))
plt.plot(p_values, prior_values, label="Prior", linestyle="--", color="blue")
plt.plot(p_values, likelihood_values, label="Likelihood", linestyle=":", color="green")
plt.plot(p_values, posterior_values, label="Posterior", linestyle="-", color="red")
plt.axvline(0.7, color="gray", linestyle="--", alpha=0.5, label="Peak of Posterior (p ≈ 0.7)")

# Annotations and Labels
plt.title("Bayesian Inference: Prior, Likelihood, and Posterior")
plt.xlabel("Probability of Heads (p)")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.show()
